In [1]:
from astropy.table import Table
import numpy as np
import pandas as pd
#from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import Planck18 as cosmo
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, AutoMinorLocator, FuncFormatter
from matplotlib.ticker import ScalarFormatter
import healpy as hp
from pylab import cm
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
import os

data=Table.read('../../../../data/quaia_G20.0.fits')
df = pd.DataFrame({
    'l': data['l'],
    'b': data['b'],
    'redshift': data['redshift_quaia']})

##########################################################################
l=df['l'].to_numpy()
b=df['b'].to_numpy()
Z=df['redshift'].to_numpy()

r=cosmo.comoving_distance(Z).value
theta = np.radians(90 - b)
phi = np.radians(l)

df1 = df = pd.DataFrame({ 'r' : r, 'theta' : theta, 'phi' : phi,
    'l': data['l'],
    'b': data['b'],
    'redshift': data['redshift_quaia']})
###########################################################################################################
for col in df1.columns:
    if df1[col].dtype.byteorder not in ('=', '|'):
        df1[col] = df1[col].astype(df1[col].dtype.newbyteorder('='))
        
df0 = df1[(df1['redshift']<=2.80) & (df1['redshift']>=0.36)]

In [2]:
# apply masking 1
def superpixel(subpix, nside_subpix, nside_superpix):
    """
    Return the indices of the super-pixels which contain each of the sub-pixels.
    """
    if nside_subpix == nside_superpix:
        return subpix
    theta, phi = hp.pix2ang(nside_subpix, subpix)
    return hp.ang2pix(nside_superpix, theta, phi)



def subpixel(superpix, nside_superpix, nside_subpix):
    """
    Return the indices of sub-pixels (resolution nside_subpix) within
    the super-pixel with (resolution nside_superpix).
    """
    if nside_superpix == nside_subpix:
        return superpix
    vec = hp.pix2vec(nside_superpix, superpix)
    radius = np.degrees(2. * hp.max_pixrad(nside_superpix))
    subpix = query_disc(nside_subpix, vec, radius=radius)
    pix_for_subpix = superpixel(subpix, nside_subpix, nside_superpix)
    # Might be able to speed up array indexing...
    return subpix[pix_for_subpix == superpix]

######################################################################################

# # === Input and output file paths ===
# f = 'quasar_sample_collapse.csv'              # single input file
# df0 = pd.read_csv(f)

# === Filter by galactic latitude b ===
fd1 = df0[(df0.b >= 40)]
fd2 = df0[(df0.b <= 0)]
fd3 = fd2[(fd2.b <= -60)]

fd4 = pd.concat([fd1, fd3])
fd4.reset_index(inplace=True, drop=True)

# === Extract columns as numpy arrays ===
r = fd4['r'].to_numpy()
th = fd4['theta'].to_numpy()
ph = fd4['phi'].to_numpy()
z = fd4['redshift'].to_numpy()

# === Define Healpix parameters ===
Nside = 64                  # nside of subpixel
Npix = hp.nside2npix(Nside)

nside = 8                   # nside of superpixel
super_pix = hp.nside2npix(nside)

# === Count sources in subpixels ===
m = np.zeros(Npix, dtype=int)
for i in range(len(r)):
    px = hp.ang2pix(Nside, th[i], ph[i])
    m[px] += 1

# === Map each subpixel to its superpixel ===
sp = np.zeros(Npix)
for j in range(Npix):
    sp[j] = superpixel(j, Nside, nside)

# === Combine subpixel ID and counts ===
data1 = np.column_stack((sp, m))
df = pd.DataFrame(data1, columns=['spp', 'mm'])

# === Find valid superpixels (>=90% filled) ===
P = []
for j in range(super_pix):
    fd = df[df.spp == j]
    fd.reset_index(inplace=True, drop=True)
    f = (len(fd[fd.mm != 0])) / len(fd)
    if f >= 0.90:
        P.append(j)

# === Save valid superpixel IDs ===
p = np.array(P)
# df1 = pd.DataFrame(data=p)
# f1 = 'non_zero_pix_id.dat'
# df1.to_csv(f1, sep='\t', header=None, index=False)

# === Save masked quasar sample ===
op = 'masked_sample.dat'
with open(op, 'w') as f2:
    for i in range(len(r)):
        px = hp.ang2pix(8, th[i], ph[i])
        if px in p:
            f2.write("%f\t%f\t%f\t%f\n" % (r[i], th[i], ph[i],z[i]))
